In [28]:
import csv
import math
import string
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, Markdown

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors as KNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Perceptron

from joblib import dump, load

from nltk.stem import PorterStemmer
import nltk
from nltk.corpus import stopwords

import gensim

import torch
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.utils.data as Data
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from torch.autograd import Variable

import utils.BoW_data as dp

In [2]:
ddir = 'data/'

In [3]:
mlb_tags_train, mlb_tags_test, df = dp.load_dataframe()

In [8]:
# Have to do this because load_dataframe() is messing up resnet_vectors somehow
with open('data/features_train/features_resnet1000_train.csv', 'r') as fp:
    features_train = [x.strip().split(',') for x in fp.readlines()]
    
features_train = {x[0]: np.array([np.float(n) for n in x[1:]]) for x in features_train}

with open('data/features_test/features_resnet1000_test.csv', 'r') as fp:
    features_test = [x.strip().split(',') for x in fp.readlines()]
    
features_test = {x[0]: np.array([np.float64(n) for n in x[1:]]) for x in features_test}

In [9]:
# Have to do this because load_dataframe() is messing up resnet_vectors somehow
with open('data/features_train/features_resnet1000intermediate_train.csv', 'r') as fp:
    int_features_train = [x.strip().split(',') for x in fp.readlines()]
    
int_features_train = {x[0]: np.array([np.float(n) for n in x[1:]]) for x in int_features_train}

# Have to do this because load_dataframe() is messing up resnet_vectors somehow
with open('data/features_test/features_resnet1000intermediate_test.csv', 'r') as fp:
    int_features_test = [x.strip().split(',') for x in fp.readlines()]
    
int_features_test = {x[0]: np.array([np.float64(n) for n in x[1:]]) for x in int_features_test}

# W2V

In [6]:
df['desc_joined'] = df['descriptions'].apply(lambda x: " ".join(x).split(" "))

In [7]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

In [10]:
ps = PorterStemmer()

stop_words = set(stopwords.words('english'))
stop_words.remove('not')

In [11]:
def get_word2vec(df_in, model):
    
    def prep_doc(doc):

        # Stem and lower all the words
        sentence = [
            ps.stem(word).lower() for word in doc
        ]

        # Remove trailing commas and periods
        sentence = [
            "".join([char for char in word if char not in string.punctuation]) for word in sentence
        ]

        # Remove stopwords, remove any blank words 
        sentence = [
            word for word in sentence if word not in stop_words and word not in  [" ", ""]
        ]
        
        return sentence
    
    def try_get_vec(word):
        try:
            word = model.get_vector(word)
            can_do = True
        except:
            word = np.zeros(300)
            can_do = False
        return can_do
    
    def get_vector(word):
        try:
            word = model.get_vector(word)
        except:
            word = np.zeros(300)
        return word
    
    stops = set(stopwords.words('english'))
    vecs = df_in['desc_joined'].apply(
        lambda desc: np.average([
            get_vector(word) for word in prep_doc(desc)
            if try_get_vec(word)
        ], axis=0)
    )
    
    return np.asarray([vec for vec in vecs], dtype=np.float64)

# BERT

In [10]:
# Return train and test dicts for looking up BERT vector from filename

bert_train = torch.load(ddir + 'bert/bert_train_avg_last_4.pt')
bert_test = torch.load(ddir + 'bert/bert_test_avg_last_4.pt')

for root, directory, files in os.walk(os.path.join('data/','descriptions_train')):
    bert_train_filenames = files
    
for root, directory, files in os.walk(os.path.join('data/','descriptions_test')):
    bert_test_filenames = files
    
bert_train_lookup = {bert_train_filenames[ind]: bert_vector.numpy() for ind, bert_vector in enumerate(bert_train)}
bert_test_lookup = {bert_test_filenames[ind]: bert_vector.numpy() for ind, bert_vector in enumerate(bert_test)}

In [12]:
df['text_file'] = df['image_file'].apply(lambda x: x.split('/')[-1].replace('jpg', 'txt'))
df.head()

,image_file,resnet_vector,descriptions,tags,tags_values,tags_vec,word_list,word_vector,desc_joined,text_file
0,images_train/5373.jpg,"[-0.8994496464729309, -0.9304700493812561, -2....","[a red train is docked at the station, Several...","[vehicle:train, person:person, indoor:clock, a...","[train, person, clock, handbag]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[red, train, dock, station, sever, peopl, stan...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[a, red, train, is, docked, at, the, station, ...",5373.txt
1,images_train/984.jpg,"[-1.3469539880752563, -3.1194605827331543, -0....",[A man with blue jersey holding a baseball bat...,"[person:person, sports:baseball bat]","[person, baseball bat]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, man, blue, jersey, hold, basebal, bat, clo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, man, with, blue, jersey, holding, a, baseb...",984.txt
2,images_train/7127.jpg,"[-3.44549822807312, -1.5245732069015503, -1.00...",[A kitchen decorated in red and white with acc...,"[appliance:refrigerator, appliance:oven, appli...","[refriger, oven, sink, cup, cake, vase]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, kitchen, decor, red, white, accessori, A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, kitchen, decorated, in, red, and, white, w...",7127.txt
3,images_train/9609.jpg,"[1.1146496534347534, -2.1671018600463867, 0.09...",[A black and white dog chasing sheep in a fiel...,"[animal:dog, animal:sheep]","[dog, sheep]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, black, white, dog, chase, sheep, field, sm...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, black, and, white, dog, chasing, sheep, in...",9609.txt
4,images_train/5293.jpg,"[1.6026496887207031, -1.5058174133300781, 3.02...",[Two bears with their mouths open in the water...,[animal:bear],[bear],"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[two, bear, mouth, open, water, A, coupl, bear...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Two, bears, with, their, mouths, open, in, th...",5293.txt


# Training / test data prep

In [13]:
# X = np.asarray([bert_train_lookup[file] for file in df['text_file']], dtype=np.float64)
X = get_word2vec(df, w2v_model)
# y = df['resnet_vector'].to_numpy()
y = np.asarray([int_features_train[x] for x in df['image_file']], dtype=np.float64)

y_true = df['image_file'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# KNN Classifier

In [ ]:
neigh = KNN(n_neighbors=3)
clf.fit(X_train, y_train)

# Random Forest Regressor

In [17]:
rfr = RandomForestRegressor(max_depth=15, random_state=0, n_estimators=100)
rfr.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
rfr.score(X_test, y_test)

# Lasso Regression

In [12]:
clf = Lasso(alpha=0.1)
clf.fit(X_train, y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [13]:
clf.score(X_test, y_test)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.006503694181384273

# Linear Regression

In [57]:
clf = LinearRegression()
clf.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [58]:
clf.score(X_test, y_test)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.1100622412332056

# SKLearn NN

At first tried with final layer at 512 but it was not converging, probably due to compression of the high dimensionality output vector which is 1000

Using final feature vecs with layers [2048, 1024] => 0.41

Need to try with intermediate feature vector, shouldnt be too hard to pull out

In [30]:
solvers = ['lbfgs', 'sgd', 'adam']

nn = MLPRegressor(
    hidden_layer_sizes=(512, 1024, 2048),
    max_iter=300,
    solver='adam',
    activation='relu',
    verbose=True
)

In [31]:
nn.fit(X_train, y_train)

Iteration 1, loss = 0.10013913
Iteration 2, loss = 0.08047096
Iteration 3, loss = 0.07151109
Iteration 4, loss = 0.06702479
Iteration 5, loss = 0.06377332
Iteration 6, loss = 0.06159315
Iteration 7, loss = 0.05994223
Iteration 8, loss = 0.05883950
Iteration 9, loss = 0.05773760
Iteration 10, loss = 0.05684320
Iteration 11, loss = 0.05618818
Iteration 12, loss = 0.05545516
Iteration 13, loss = 0.05482878
Iteration 14, loss = 0.05413676
Iteration 15, loss = 0.05355236
Iteration 16, loss = 0.05305961
Iteration 17, loss = 0.05266467
Iteration 18, loss = 0.05218153
Iteration 19, loss = 0.05162271
Iteration 20, loss = 0.05096629
Iteration 21, loss = 0.05042480
Iteration 22, loss = 0.04998248
Iteration 23, loss = 0.04942270
Iteration 24, loss = 0.04900718
Iteration 25, loss = 0.04854048
Iteration 26, loss = 0.04796030
Iteration 27, loss = 0.04756818
Iteration 28, loss = 0.04722488
Iteration 29, loss = 0.04682627
Iteration 30, loss = 0.04635877
Iteration 31, loss = 0.04583641
Iteration 32, los

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(512, 1024, 2048), learning_rate='constant',
             learning_rate_init=0.001, max_iter=300, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=True, warm_start=False)

In [49]:
cvnn = cross_val_score(
    nn,
    X,
    y, 
    cv=3
)

Iteration 1, loss = 0.10213039
Iteration 2, loss = 0.08289681
Iteration 3, loss = 0.07352271
Iteration 4, loss = 0.06841964
Iteration 5, loss = 0.06490570
Iteration 6, loss = 0.06271097
Iteration 7, loss = 0.06090369
Iteration 8, loss = 0.05970405
Iteration 9, loss = 0.05848295
Iteration 10, loss = 0.05753701
Iteration 11, loss = 0.05669773
Iteration 12, loss = 0.05605665
Iteration 13, loss = 0.05534638
Iteration 14, loss = 0.05468103
Iteration 15, loss = 0.05414915
Iteration 16, loss = 0.05361813
Iteration 17, loss = 0.05305640
Iteration 18, loss = 0.05247406
Iteration 19, loss = 0.05196433
Iteration 20, loss = 0.05138178
Iteration 21, loss = 0.05087186
Iteration 22, loss = 0.05031324
Iteration 23, loss = 0.04983056
Iteration 24, loss = 0.04954192
Iteration 25, loss = 0.04884425
Iteration 26, loss = 0.04843693
Iteration 27, loss = 0.04820687
Iteration 28, loss = 0.04747635
Iteration 29, loss = 0.04711037
Iteration 30, loss = 0.04662736
Iteration 31, loss = 0.04619926


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


Iteration 1, loss = 0.10160625
Iteration 2, loss = 0.08326778
Iteration 3, loss = 0.07366414
Iteration 4, loss = 0.06833266
Iteration 5, loss = 0.06528216
Iteration 6, loss = 0.06289127
Iteration 7, loss = 0.06115513
Iteration 8, loss = 0.05987916
Iteration 9, loss = 0.05876723
Iteration 10, loss = 0.05801181
Iteration 11, loss = 0.05697883
Iteration 12, loss = 0.05619124
Iteration 13, loss = 0.05547826
Iteration 14, loss = 0.05493970
Iteration 15, loss = 0.05432136
Iteration 16, loss = 0.05374978
Iteration 17, loss = 0.05314637
Iteration 18, loss = 0.05282269
Iteration 19, loss = 0.05209716
Iteration 20, loss = 0.05157417
Iteration 21, loss = 0.05112827
Iteration 22, loss = 0.05051230
Iteration 23, loss = 0.05009210
Iteration 24, loss = 0.04955807
Iteration 25, loss = 0.04914711
Iteration 26, loss = 0.04856760
Iteration 27, loss = 0.04819530
Iteration 28, loss = 0.04762742
Iteration 29, loss = 0.04721604
Iteration 30, loss = 0.04675607
Iteration 31, loss = 0.04634193
Iteration 32, los

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


Iteration 1, loss = 0.10322013


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [50]:
cvnn

array([0.34693251, 0.32064275, 0.00282394])

In [32]:
nn.score(X_test, y_test)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.09424469554256429

In [39]:
dump(nn, 'relu38percent.joblib') 

['relu38percent.joblib']

12:35 - 0.5231418465059838

Adam  - 0.25864671144762147

# Pytorch NN

In [56]:
class Net(torch.nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.hidden1 = torch.nn.Linear(300, 512)
        self.hidden2 = torch.nn.Linear(512, 1024)
        self.predict = torch.nn.Linear(1024, 1000)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.predict(x) 
        return x


net = Net()
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()

BATCH_SIZE = 64
EPOCH = 200

torch_dataset = Data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))

loader = Data.DataLoader(
    dataset=torch_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)

Net(
  (hidden1): Linear(in_features=300, out_features=512, bias=True)
  (hidden2): Linear(in_features=512, out_features=1024, bias=True)
  (predict): Linear(in_features=1024, out_features=1000, bias=True)
)


In [57]:
for epoch in range(EPOCH):
    
    for step, (batch_x, batch_y) in enumerate(loader):

        b_x = Variable(batch_x)
        b_y = Variable(batch_y)

        prediction = net(b_x)

        loss = loss_func(prediction, b_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #2 'mat1' in call to _th_addmm

# Model Validation

In [33]:
model = nn

In [34]:
def score_prediction(true, pred):

    if true not in pred:
        return 0.0
    
    ind = pred.index(true)
    
    return (20 - ind) / 20

In [35]:
def nearest_neighbors(sample, targets):
    """Returns index positions of nearest neighbors"""
    distances = [np.linalg.norm(t-sample) for t in targets]
    distances_sorted = np.argsort(distances)
    return distances_sorted[:20]

In [36]:
predicted_fvs = [model.predict(desc.reshape(1, -1)) for desc in X_test]
pred_indices = [nearest_neighbors(sample, y_test) for sample in predicted_fvs]

preds = [
    [y_true[ind] for ind in pred]
    for pred in pred_indices
]

# score = sum([1 if y_true[ind] in pred else 0 for ind, pred in enumerate(preds)]) / len(preds)

In [37]:
score = sum(
    [score_prediction(y_true[ind], pred) for ind, pred in enumerate(preds)]
           ) / len(preds)

In [38]:
print(score)

0.3896250000000002


12:35 score - 0.37332099999999513

random forest - 0.0018503043118560125

# Making predictions

In [40]:
model = nn

In [41]:
mlb_tags_train2, mlb_tags_test2, df_test = dp.load_dataframe(train_or_test='test')

df_test['desc_joined'] = df_test['descriptions'].apply(lambda x: " ".join(x).split(" "))

In [42]:
df_test.head()

,image_file,resnet_vector,descriptions,tags,tags_values,tags_vec,word_list,word_vector,desc_joined
0,images_test/152.jpg,"[-0.148756742477417, -0.4813389182090759, -0.6...","[A sign sitting above a store front entrance.,...",[person:person],[person],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, sign, sit, store, front, entranc, small, a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, sign, sitting, above, a, store, front, ent..."
1,images_test/901.jpg,"[-2.7743351459503174, -2.609677791595459, -2.2...",[A steam locamotive passes by some houses on a...,[indoor:clock],[clock],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, steam, locamot, pass, hous, track, A, smal...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, steam, locamotive, passes, by, some, house..."
2,images_test/1609.jpg,"[-4.7030134201049805, -3.1210086345672607, -0....",[a woman with purple hair is taking a picture ...,"[animal:dog, animal:horse, person:person]","[dog, hors, person]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[woman, purpl, hair, take, pictur, A, hipster,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[a, woman, with, purple, hair, is, taking, a, ..."
3,images_test/501.jpg,"[-4.7203192710876465, -3.283935546875, -3.4253...",[A GREEN AND BROWN SUITCASE LYING UP ON A LEDG...,[furniture:toilet],[toilet],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, green, and, brown, suitcas, ly, UP, ON, A,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, GREEN, AND, BROWN, SUITCASE, LYING, UP, ON..."
4,images_test/517.jpg,"[-2.5421407222747803, 0.7139833569526672, -4.4...",[A desk with a small white computer set up on ...,"[animal:cat, kitchen:bowl, furniture:bed]","[cat, bowl, bed]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, ...","[A, desk, small, white, comput, set, A, desk, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, desk, with, a, small, white, computer, set..."


In [43]:
test_descriptions = get_word2vec(df_test, w2v_model)

y_resnet = np.asarray([int_features_test[x] for x in df_test['image_file']], dtype=np.float64)
y_targets = df_test['image_file'].to_numpy()

In [44]:
predicted_fvs = [model.predict(desc.reshape(1, -1)) for desc in test_descriptions]
pred_indices = [nearest_neighbors(sample, y_resnet) for sample in predicted_fvs]

preds = [
    [y_targets[ind] for ind in pred]
    for pred in pred_indices
]

preds = [[file.split('/')[-1] for file in pred] for pred in preds]

In [45]:
desc_txt_files = df_test['image_file'].apply(
        lambda x: x.split('/')[-1].replace('jpg', 'txt')
    )

In [46]:
out_preds = list(zip(desc_txt_files.to_numpy(), preds))

In [47]:
with open('12_10_preds.csv', 'w+') as f:
    f.write('Descritpion_ID,Top_20_Image_IDs\n')
    for pred in out_preds:
        f.write(f"{pred[0]},{' '.join(pred[1])}\n")